### Importación de bibliotecas necesarias

In [5]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re


### Consultar Wikidata

In [ ]:

def limpiar_nombre(nombre):
    """Limpia nombres de artistas eliminando caracteres problemáticos para consultas SPARQL.

    Args:
        nombre: Nombre del artista como cadena de texto (str o NaN).

    Returns:
        str o None: Nombre limpio o None si está vacío o es NaN.
    """
    if pd.isna(nombre) or not nombre.strip():
        return None
   
    nombre = nombre.replace("\\", "")   
    nombre = nombre.replace('"', '')  
    nombre = nombre.replace("'", "")    
    nombre = nombre.replace("/", " ")   
    nombre = nombre.replace("&", "and") 
    nombre = nombre.strip()
    return nombre


df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))


print(f"Total artistas únicos: {len(artistas_unicos)}")


WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "ETL-MusicalProject/1.0 (tu_correo_ejemplo@gmail.com)"  
}

def construir_query_sparql(artistas):
    """Construye una consulta SPARQL para obtener datos de artistas desde Wikidata.

    Args:
        artistas: Lista de nombres de artistas (list).

    Returns:
        str: Consulta SPARQL como cadena de texto.
    """
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?death ?countryLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    """Realiza una consulta a Wikidata y devuelve los resultados en formato JSON.

    Args:
        artistas_batch: Lista de nombres de artistas en el batch (list).

    Returns:
        dict o None: Resultados en JSON o None si hay un error.
    """
    query = construir_query_sparql(artistas_batch)
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error en SPARQL:", e)
        return None


print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000  
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80 
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5  
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(0.8)  
            else:
                batch_size = batch_size // 2  

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)


columnas_ordenadas = ["artist", "country", "award", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)


Total artistas únicos: 24063
🚀 Consultando Wikidata...


🔎 Batches SPARQL:  75%|███████▌  | 18160/24063 [08:50<02:37, 37.56it/s]

### Dividir y guardar el DataFrame en dos partes

In [4]:

halfway = len(df_result) // 2


df_result.iloc[:halfway].to_csv("../data/wikidata_artists_part1.csv", index=False)
print("✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.")

df_result.iloc[halfway:].to_csv("../data/wikidata_artists_part2.csv", index=False)
print("✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.")

✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.
✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.
